In [1]:
import pandas as pd
import re

In [2]:
jsonList = pd.read_json('/content/task_input_list.json')

In [3]:
 df = jsonList[0]

In [4]:
for i in range(len(jsonList[0])):
  if jsonList[0][i] == 'Deposits and other additions':
    DepositIndex = i+1
  if jsonList[0][i] == 'Other subtractions':
    withdrawalIndex = i+1
  if jsonList[0][i]=='Total other subtractions':
    withdrawalend= i

In [5]:
columns = [jsonList[0][i] for i in range(DepositIndex,DepositIndex+3)]
columns

['Date', 'Description', 'Amount']

In [6]:
def transaction(df, start, end):

  import re
  def check_date(date_):
    
    if re.match('\d{2}(?P<sep>[-/])\d{2}(?P=sep)\d{2}',date_):
      return True
    else:
      return False

  def check_amount(amt):
  
    try:
      if ',' in amt:
        idx = amt.index(',')
        amt = amt[0:idx] + amt[(idx+1):]   
        
      if float(amt):
        return float(amt)
      
    except:
      return False

  def check_description(desc):
    
    if not check_amount(desc) and not check_date(desc):
      return True

  def parse_data(df, start, end):
    date= []
    day=[]
    month=[]
    year=[]
    amountList =[]
    desc = []
    for i in range(start,end):
 
      if check_date(df[i]):

        date.append(df[i])
        d = df[i].split('/')
        day.append(d[1])
        month.append(d[0])
        year.append(int(d[2])+2000)
      amount = check_amount(df[i])

      if amount:
        
        amountList.append(amount)

      if check_description(df[i]):
       
        if check_description(df[i-1]):
          desc[-1] += ' ' + df[i]
        else:
          desc.append(df[i])

    res = pd.DataFrame(columns=['DATE','DESCRIPTION','AMOUNT','DAY','MONTH','YEAR'])

    res['DATE'] = date
    res['DESCRIPTION'] = desc
    res['AMOUNT'] = amountList
    res['DAY'] = day
    res['MONTH'] = month
    res['YEAR'] = year

    
   
    return res
  result = parse_data(df, start, end)
  return result


In [7]:
deposits = transaction(df, DepositIndex + 3, withdrawalIndex - 4)

In [8]:
print(deposits)

       DATE                                    DESCRIPTION  ...  MONTH  YEAR
0  12/03/18                                 Counter Credit  ...     12  2018
1  12/10/18                                 Counter Credit  ...     12  2018
2  12/21/18  PAYPAL DES:TRANSFER ID:1004561753452 INDN: CO  ...     12  2018
3  12/24/18                                 Counter Credit  ...     12  2018

[4 rows x 6 columns]


In [9]:
withdrawals = transaction(df, withdrawalIndex + 3, withdrawalend)

In [10]:
withdrawals

,DATE,DESCRIPTION,AMOUNT,DAY,MONTH,YEAR
0,11/30/18,Automatic Transfer to SAV 2084 Confirmation# 1...,-25.00,30,11,2018
1,12/03/18,Online Banking payment to CRD 8610 Confirmatio...,-200.00,03,12,2018
2,12/04/18,Customer Withdrawal Image,-12000.00,04,12,2018
3,12/07/18,TRANSFER,-3695.00,07,12,2018
4,12/11/18,CARFAX DES:BT1210 ID:000000071591462 INDN: CO ...,-399.98,11,12,2018
5,12/13/18,TRANSFER :J Cars Confirmation# 0472167145,-3955.00,13,12,2018
6,12/19/18,TRANSFER :J Cars Confirmation# 0423610629,-13990.00,19,12,2018
7,12/20/18,SOUTHWEST GAS DES:WEB ID:1410023972050 INDN:JO...,-28.25,20,12,2018
8,12/21/18,TRANSFER :J Cars Confirmation# 1642822709,-16630.00,21,12,2018


In [11]:
def insights(df, deposits, withdrawals):
  def check_website(text):
    pattern = r'[0-9a-z]{0,61}\.[a-z]{2,4}$'
    if re.match(pattern, text):
      return True
    else:
      return False

  def check_email(email):
  
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    if re.match(pattern, email):
      return True
    else:
      return False

  def check_phno(number):
    pattern = r"^[1]*[-]*[(]*[0-9]{3}[)]*[-]*[ ]*[0-9]{3}[-]*[0-9]{4}"
    if re.match(pattern, number):
      return True
    else:
      return False

  def parse_data(df, deposits, withdrawals):
    phone_number=[]
    emailList = []
    website = []

    for i in range(len(df)):
      if ": " in df[i]:
        ph = df[i].split(": ")[1]
        
        if check_phno(ph):
          phone_number.append(ph)

      if check_email(df[i]):
        emailList.append(df[i])
      if check_website(df[i]):
        website.append(df[i]) 


    

    phone = ','.join(phone_number) if phone_number else 'NA'
    emailIds = ','.join(emailList) if emailList else 'NA'
    websites = ','.join(website) if website else 'NA'
    maxAmount = max(map(max,[deposits['AMOUNT'],withdrawals['AMOUNT']]))
    minAmount = min(map(min,[deposits['AMOUNT'],withdrawals['AMOUNT']]))

    res = pd.DataFrame(columns=['Key',"Value"])
    res['Key'] = ['website', 'email', 'phone_numbers', 'max amount', 'min amount']
    res['Value'] = [websites, emailIds, phone, maxAmount, minAmount]
    
    return res

  result = parse_data(df, deposits, withdrawals)
  return result



In [12]:
insights = insights(df, deposits, withdrawals)

In [13]:
insights

,Key,Value
0,website,bankingservices.com
1,email,NA
2,phone_numbers,"(555) 555-1234,(800) 555-1234,1-800-555-1234"
3,max amount,9970
4,min amount,-16630


In [14]:
with pd.ExcelWriter('task_2_output.xlsx') as writer:  
    deposits.to_excel(writer, sheet_name='DEPOSITS', index=False)
    withdrawals.to_excel(writer, sheet_name='WITHDRAWALS', index=False)
    insights.to_excel(writer, sheet_name='INSIGHTS', index=False)